In [46]:
require('nn')
require('hdf5')
require('optim')

In [47]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [64]:
function bigramCount(X, K)
    local countMatrix = torch.zeros(K, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local prev_word = X[i][j-1]
            local curr_word = X[i][j]
            countMatrix[prev_word][curr_word] = countMatrix[prev_word][curr_word] + 1
        end
    end
    return countMatrix
end

function unigramCount(X, Y, K)
    local countVector = torch.zeros(1, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local idx = X[i][j]
            countVector[1][idx] = countVector[1][idx] + 1
        end
    end
    return countVector
end
 

In [58]:
function additive(matrix, alpha)
    return matrix:add(alpha)
end

function normalize(CV, CM)
    return CM:cdiv(torch.expand(CV:t(), CM:size(1), CM:size(2)))
end

function test(bcm, valid, kaggle, q)
    p = 0
    for i=1,valid:size(1) do
        idx = q[i][kaggle[i]]
        prev = valid[i][5]
        p = torch.log(bcm[prev][idx]) + p
    end
    return torch.exp(-p/X_valid:size(1))
end

function kaggle(bcm, test, q)
    scores = torch.zeros(q:size(1), 50)
    for i=1,test:size(1) do
        prev = test[i][5]
        for j=1,50 do
            w = q[i][j]
            scores[i][j] = bcm[prev][w]
        end
    end
    return scores:cdiv(torch.expand((torch.sum(scores, 2)), scores:size(1), scores:size(2)))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [65]:
CM = bigramCount(X_train, nfeatures)
CM = additive(CM, 1)

In [66]:
CV = unigramCount(X_train, Y_train, nfeatures)
CV = additive(CV, 1)

In [68]:
BCM = normalize(CV, CM)

In [ ]:
JLL = distribution(BCM, X_valid)

In [55]:
valid_perplexity(JLL, kaggle_valid, q_valid)

-33413.805373877	
20233.112593302	



In [41]:
scores = kaggle(BCM, X_test, q_test)

In [42]:
write2file(scores, "asdf.csv")

In [69]:
test(BCM, X_valid, kaggle_valid, q_valid)

133.14713426227	


In [67]:
CM:sub(2,2)

Columns 1 to 11
  5562  14426      1      1   1089   6071   3586   3724   3991   7393   1830

Columns 12 to 22
  1667   1891    299   1573    511   1594   1313   1829    959   1007    966

Columns 23 to 33
     6   1614    660    849      9    691    479    367    497    587    997

Columns 34 to 44
     1    655    405    180    745    201    184    189     37    661    281

Columns 45 to 55
   517    704    196    366    205      5    387    375    371    595    107

Columns 56 to 66
   194    297   1203    213     13    221    109     62     21    115    109

Columns 67 to 77
   657    179    280    171    123     92    516     43    142     70     90

Columns 78 to 88
   115    253    124    197    697    165    738     70    502    267    109

Columns 89 to 99
    71    164    119     72    442      5    157    353    268    413    109

Columns 100 to 110
    62     97     67    127     53     34    110    119     95      1     86

Columns 111 to 121
    22     79     16    107   

   27      1     25     30     13     30     33     10

Columns 474 to 484
    15      5     13     42     51     58     15     13      1     86     33

Columns 485 to 495
    55     21     12     39     61      1     25     33      7     28     25

Columns 496 to 506
    32     11     16     33      9     42     25      5     90     13     38

Columns 507 to 517
     6      9     17     45    118      5     61     21     13     61     22

Columns 518 to 528
    17     21     25      6     38      5     28      1     17     44     30

Columns 529 to 539
    13     61     48     10     39     35     26     34     29     15     60

Columns 540 to 550
    31     99     25      5     52     41     52     29     17      1     59

Columns 551 to 561
    63     12     45    185     39    112     40     57     11     41      4

Columns 562 to 572
    94     13     28     35     54      9      9     19     13     37     31

Columns 573 to 583
    21     17      9      9      9      1      1    

   1      2      3     36      5

Columns 936 to 946
    25     14     20      9      1     41     24     25     41     35     41

Columns 947 to 957
    21     33     11     26     13     13     15      5     12     21      9

Columns 958 to 968
    69     54      9      5     13      5      9     11      6      5     15

Columns 969 to 979
     6      5     19     34      5      5     15     26      5     13     31

Columns 980 to 990
     9     13      1      5      9     48     29     13     13     21     17

Columns 991 to 1001
   129     35     45     29      1      1     23      5     21     15     17

Columns 1002 to 1012
    17     12     18      5      1      9     20     21     29      1      1

Columns 1013 to 1023
     1      9     27      9     25     33     13     20      5      5     10

Columns 1024 to 1034
    11      5     28      9     13     18      1      9     13     10      3

Columns 1035 to 1045
    34      6     22     14     25      8      5     21     65   

o 1397
     1     25     10      5      1     21     12     10      1      9      1

Columns 1398 to 1408
     9     13     13     25      1     62     15      9     28      1     35

Columns 1409 to 1419
     5      1      9      1     31      3      2      9      7     13      1

Columns 1420 to 1430
    21     17     21      1     23      1      9     13      9      5     14

Columns 1431 to 1441
    16      1      9     13      1     12      1     13      5      8      1

Columns 1442 to 1452
     5      2      1     19      5     21     13     26     13     11      9

Columns 1453 to 1463
    21     17     17      9      5      3      2     22     16     21      5

Columns 1464 to 1474
    18      5      5      1      1      5      1     35     15      1      8

Columns 1475 to 1485
    17     57      5     14      5      3     10     10      9     13      5

Columns 1486 to 1496
     6     13     17     13     37      9      1      7      2     18     11

Columns 1497 to 1507
   

  10      5      1     29     13     17     25

Columns 1849 to 1859
    97      6      1     13      1     13     11     17      9      9      5

Columns 1860 to 1870
     6      1     10      1     21      1      1      9     17     26      5

Columns 1871 to 1881
     1      5     11      9      5     13     12     18     13      8      1

Columns 1882 to 1892
    32      1      5      1      9      1     21     25      9      9      1

Columns 1893 to 1903
    13      1     13     15     12      5      1      1     13     20      5

Columns 1904 to 1914
    13     24     10      1      1      1     13      5      1      1      1

Columns 1915 to 1925
    13      1      9     22      5     13      9      1      7      1      1

Columns 1926 to 1936
    21     17      5      1      5      5      9      6      5      5      9

Columns 1937 to 1947
     5      5     18      9      9      5      5     17      5     17      1

Columns 1948 to 1958
     1     31      5      1      5     1

...

   5      1      2      5      5      1      5      1      1

Columns 7767 to 7777
     1      1      5      9      5      1      1      1      1      1     13

Columns 7778 to 7788
     1      1      1      9      1      1      1      5      5      1      1

Columns 7789 to 7799
     1      1      1      5      1      1      9      5      1      1      5

Columns 7800 to 7810
     5      1      1      1      1      1      1      1      7      1      6

Columns 7811 to 7821
     1      1      5      1      1      5      9     29      1      1      1

Columns 7822 to 7832
     5      5      1      1      1      1      9      5      5      1      6

Columns 7833 to 7843
     1      1      5      1      1      5      5      1      1      9      3

Columns 7844 to 7854
     1      1      1      1     13      2      5      4      9      1      1

Columns 7855 to 7865
    17      1      1      1      1      5      1      5      1      1      1

Columns 7866 to 7876
     5      2      1      

 1      1      1      9

Columns 8218 to 8228
     5      5      5      1      1      1      1      5      1     10      5

Columns 8229 to 8239
     1      1      9      5      1      5      1      1      1      1      5

Columns 8240 to 8250
     5      1      1      1      1      1      1      1      1      5      5

Columns 8251 to 8261
     5      3      1      1      5      1      1      1      1      1      1

Columns 8262 to 8272
     1      1      2      1      1      1      7      5      1      1      5

Columns 8273 to 8283
     1      9      5      9      1      5      5      1      1      5      1

Columns 8284 to 8294
     2      1      5      1      2      1      1      1      1      9      1

Columns 8295 to 8305
     9      5      1      5      1      5      1      1      1      1      5

Columns 8306 to 8316
     5      3      1     17      1      1      5      1      5      1      1

Columns 8317 to 8327
     5      9      1      1      5      1      1      2      1 

 to 8679
     1      1      1      1      1      1      5      1      5      5      1

Columns 8680 to 8690
     1      5      5      1      1      1      1      1      1      1      1

Columns 8691 to 8701
     1      1      1      6      1      1      1      1      9      1      5

Columns 8702 to 8712
     5      1      1      9      1      1      1      7      5      1      1

Columns 8713 to 8723
     1      1      5      1     22      1      1      1      1      1      1

Columns 8724 to 8734
     5      1      1      1      1      1      1      5      1      1      1

Columns 8735 to 8745
     1      1      1      1      1      1      4      1     13      1      1

Columns 8746 to 8756
     1     15      1      1      1      9      1      1      1      5      6

Columns 8757 to 8767
     1      1      5      1      1      1      1      1      1      1      1

Columns 8768 to 8778
     1      1      3      1     13      1      1      1      1      1      1

Columns 8779 to 8789
 

     1      1      1      1      1      5      1

Columns 9131 to 9141
     1      5      1      4      1      5      1     17      1      1      1

Columns 9142 to 9152
     1      1      1      5      2      1      1      1      5      1      1

Columns 9153 to 9163
     1     17      1      5      1      1      5      1      5      1      1

Columns 9164 to 9174
     1      1      1      1      1      1      1      5      3      1      5

Columns 9175 to 9185
     2      5      5      1      1      1      1      1      1      1      1

Columns 9186 to 9196
     1      1      5      2      1      1      1      5      1      1      1

Columns 9197 to 9207
     1      1      1      1      5      1      1      1      1     16      5

Columns 9208 to 9218
     5      1      1      1      1      1      5      1      1      1      1

Columns 9219 to 9229
     1      1      1      1      1      9      1      5     21      5      1

Columns 9230 to 9240
     1      1      5     16      1    

   1      1

Columns 9582 to 9592
     1      1      5      5      5      5      1      1      1      1      1

Columns 9593 to 9603
     1      1      1      5      1      1      1      1      1      1      1

Columns 9604 to 9614
     5      1      3      1      1      1      1      1      5      1      1

Columns 9615 to 9625
     1      1      1      1      1      1      1      9      5      6      5

Columns 9626 to 9636
     1      1      1      1      7      1      5      1      1      1      5

Columns 9637 to 9647
     5      1      1      5      1      1      5      5      1      1      1

Columns 9648 to 9658
     5      9      1      1      5      6      1      1      5      5      1

Columns 9659 to 9669
     1      6      1      5      1      1      5      1      1      1      5

Columns 9670 to 9680
    13      1      1      1      1      1      1      1      9      1      1

Columns 9681 to 9691
     1      1      1      1      1      1      1      1      1      5      

In [45]:
scores:sub(3,3)

Columns 1 to 10
 0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009

Columns 11 to 20
 0.0009  0.0009  0.9543  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009

Columns 21 to 30
 0.0043  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009

Columns 31 to 40
 0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009

Columns 41 to 50
 0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009  0.0009
[torch.DoubleTensor of size 1x50]

